In [ ]:
import numpy as np

vec = np.load("../data/hog/0.npy")
print(vec.shape)
print(vec[:20])


(8100,)
[0.29169273 0.17284098 0.07630757 0.10154331 0.15816405 0.02941237
 0.15900024 0.05533042 0.21555001 0.06538779 0.11610029 0.02647543
 0.1133854  0.32852079 0.13308106 0.1288689  0.05517796 0.04610085
 0.09782295 0.04713698]
